In [ ]:
import geopandas as gpd
import pandas as pd
import pypsa

In [ ]:
gdf = gpd.read_file("Geospatial/regions_onshore_s20_4m.geojson").set_index("name")
gdf["p"] = gdf.geometry.centroid
gdf.plot()

In [ ]:
n = pypsa.Network("elec_s20_c4m_ec_lv1.0_8760SEG_E-G.nc")

In [ ]:
df = n.loads[n.loads.carrier.str.startswith("res")].copy()
df["cluster"] = df.bus.map(lambda x: x.split(" res")[0])
loads = df.index.to_list()
load_map = df["cluster"].to_dict()
clusters = list(set([x for _, x in load_map.items()]))

In [ ]:
summed_loads = n.loads_t.p_set[df.index].rename(columns=load_map).T.groupby(level=0).sum().T
summed_loads = summed_loads.sum().to_frame(name="load")
l = gdf.join(summed_loads)

In [ ]:
l.p.iloc[0].coords[0]

In [ ]:
import matplotlib.pyplot as plt

fig, axd = plt.subplot_mosaic(
    [["left", "top_mid", "top_right"], ["left", "bot_mid", "bot_right"]], figsize=(16, 8), layout="constrained"
)

l.plot("load", ax=axd["left"], legend=True, legend_kwds={"label": "Yearly Energy Load (MWh)", "location": "left"})

for idx, row in l.iterrows():
    axd["left"].annotate(text=idx, xy=row["p"].coords[0], horizontalalignment="center")

carrier_map = {"elec": "Electricity", "space-heat": "Space Heating", "water-heat": "Water Heating", "cool": "Cooling"}

for ax_name, carrier in zip(
    ["top_mid", "top_right", "bot_mid", "bot_right"], ["elec", "space-heat", "water-heat", "cool"]
):
    df1 = n.loads_t.p_set[[x for x in loads if x.endswith(carrier)]]
    df1 = df1.rename(columns={x: x.split(" ")[0] for x in df1.columns})
    df1 = df1.T.groupby(level=0).sum().T
    df1.loc[2030].resample("D").mean().plot(title=carrier_map[carrier], ax=axd[ax_name], xlabel="")
    axd[ax_name].legend(loc="upper right", title="Cluster")

fig.savefig("clusterd-load.png", dpi=300)

In [ ]:
import seaborn as sns

In [ ]:
def get_hp_cop(n):
    cops = n.links_t.efficiency

    ashp = cops[[x for x in cops.columns if x.endswith("ashp")]]
    gshp = cops[[x for x in cops.columns if x.endswith("gshp")]]

    return ashp.join(gshp)

In [ ]:
def plot_hp_cop(n: pypsa.Network, state: str = None, **kwargs) -> tuple:
    """
    Plots gshp and ashp cops.
    """

    investment_period = n.investment_periods[0]

    cops = get_hp_cop(n).loc[investment_period]
    cops.index = cops.index.map(lambda x: x.replace(year=2018))

    fig, axs = plt.subplots(
        nrows=1,
        ncols=2,
        figsize=(16, 6),
        sharey=True,
    )

    mapper = {"ashp": "Air Source Heat Pump", "gshp": "Ground Source Heat Pump"}

    for i, hp in enumerate(["ashp", "gshp"]):

        df = cops[[x for x in cops if x.endswith(hp)]]
        avg = df.mean(axis=1)

        palette = sns.color_palette(["lightgray"], df.shape[1])

        try:

            sns.lineplot(
                df,
                color="lightgray",
                legend=False,
                palette=palette,
                ax=axs[i],
            )
            sns.lineplot(avg, ax=axs[i])

            axs[i].set_xlabel("")
            axs[i].set_ylabel("COP")
            axs[i].set_title(mapper[hp])

        except TypeError:  # no numeric data to plot
            # logger.warning(f"No COP data to plot for {state}")
            pass

    return fig, axs

In [ ]:
fig, ax = plot_hp_cop(n)
fig.tight_layout()
fig.savefig("COP", dpi=300)